# Further Pre-training MobileBERT MLM with Federated Averaging (Shakepeare)

In [1]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Google Colab settings

In [1]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    # Mount Google Drive root directory
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

### CUDA Multi GPU

In [2]:
# Use this code snippet to use specific GPUs
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [3]:
# IPython reloading magic
%load_ext autoreload
%autoreload 2

In [4]:
# Install required packages
# !pip install -r requirements.txt

## Import packages

In [5]:
import tensorflow as tf

In [6]:
tf_physical_devices_gpu = tf.config.list_physical_devices('GPU')

# Allow the growth of GPU memory consumption to take place incrementally
if tf_physical_devices_gpu:
    for gpu in tf_physical_devices_gpu:
        tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
import os
import sys
import random
import datetime
import json
import pathlib
import itertools
import time

import numpy as np
import tensorflow_federated as tff
import tensorflow_text as tf_text
import tensorflow_addons as tfa
import transformers

import nest_asyncio
nest_asyncio.apply()

import fedavg
import fedavg_client
import datasets
import utils


# Random seed settings
random_seed = 692
random.seed(random_seed) # Python
np.random.seed(random_seed) # NumPy
tf.random.set_seed(random_seed) # TensorFlow

# Test if TFF is working
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [8]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
NumPy version: 1.18.4
TensorFlow version: 2.3.1
TensorFlow Federated version: 0.17.0
Transformers version: 3.4.0


In [9]:
!nvidia-smi

Fri Nov 27 21:23:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    On   | 00000000:03:00.0 Off |                  N/A |
|  0%   38C    P2    46W / 205W |    231MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
tf_logical_devices_cpu = tf.config.list_logical_devices('CPU')
tf_logical_devices_gpu = tf.config.list_logical_devices('GPU')

## Experiment Settings

In [11]:
EXPERIMENT_CONFIG = {}

EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'] = 'google/mobilebert-uncased'
EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'] = os.path.join('.', 'transformers_cache')

EXPERIMENT_CONFIG['TOTAL_ROUNDS'] = 1000 # Number of total training rounds
EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] = 1 # How often to evaluate

EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'] = 10 # How many clients to sample per round.
EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'] = 3

EXPERIMENT_CONFIG['BATCH_SIZE'] = 8 # Batch size used on the client.
EXPERIMENT_CONFIG['TEST_BATCH_SIZE'] = 32 # Minibatch size of test data.

# Maximum length of input token sequence for BERT.
EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'] = 128

# Optimizer configuration
EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'] = 1.0 # Server learning rate.
EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'] = 3e-5 # Client learning rate

# Client dataset setting
EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] = -1
EXPERIMENT_CONFIG['TEST_NUM_CLIENT_LIMIT'] = -1

# Path to save trained weights and logs
EXPERIMENT_CONFIG['RESULTS_DIRECTORY'] = os.path.join(
    '.', 'results',
    'mobilebert_mlm_shakespeare_fedavg',
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

EXPERIMENT_CONFIG['RESULTS_LOG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "logs")
EXPERIMENT_CONFIG['RESULTS_MODEL'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "model")
EXPERIMENT_CONFIG['RESULTS_CONFIG'] = os.path.join(EXPERIMENT_CONFIG['RESULTS_DIRECTORY'], "config")

In [12]:
# Dump all the configuration into a json file
pathlib.Path(EXPERIMENT_CONFIG['RESULTS_CONFIG']).mkdir(parents=True, exist_ok=True)

with open(os.path.join(EXPERIMENT_CONFIG['RESULTS_CONFIG'], "config.json"), 'w') as config_file:
    json.dump(EXPERIMENT_CONFIG, config_file, indent=6)

In [13]:
# TFF executor factory settings
# Reference: https://www.tensorflow.org/federated/api_docs/python/tff/backends/native/set_local_execution_context
tff.backends.native.set_local_execution_context(
    num_clients=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
    max_fanout=100,
    clients_per_thread=1,
    server_tf_device=tf_logical_devices_cpu[0],
    client_tf_devices=tf_logical_devices_cpu,
)

## Dataset

### Dataset loader

In [14]:
train_client_data, test_client_data = tff.simulation.datasets.shakespeare.load_data(cache_dir='./tff_cache')

### Tokenizer

In [15]:
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

In [16]:
# Imitate transformers tokenizer with TF.Text Tokenizer
tokenizer_tf_text, vocab_lookup_table, special_ids_mask_table = \
datasets.preprocessing_for_bert.convert_huggingface_tokenizer(bert_tokenizer)

### Preprocessing

In [17]:
def check_empty_snippet(x):
    return tf.strings.length(x['snippets']) > 0

def tokenizer_and_mask_wrapped(x):

    masked, labels = datasets.preprocessing_for_bert.tokenize_and_mask(tf.reshape(x['snippets'], shape=[1]),
                                                                       max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
                                                                       bert_tokenizer_tf_text=tokenizer_tf_text,
                                                                       vocab_lookup_table=vocab_lookup_table,
                                                                       special_ids_mask_table=special_ids_mask_table,
                                                                       cls_token_id=bert_tokenizer.cls_token_id,
                                                                       sep_token_id=bert_tokenizer.sep_token_id,
                                                                       pad_token_id=bert_tokenizer.pad_token_id,
                                                                       mask_token_id=bert_tokenizer.mask_token_id)

    return (masked, labels)

def preprocess_for_train(train_dataset):
    return (
        train_dataset
        # Tokenize each samples using MobileBERT tokenizer
        #.map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=20, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly BATCH_SIZE
        # and make the shape **exactly** (BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['BATCH_SIZE'])#, drop_remainder=True)
        # Repeat to make each client train multiple epochs
        .repeat(count=EXPERIMENT_CONFIG['CLIENT_EPOCHS_PER_ROUND'])
    )
    
def preprocess_for_test(test_dataset):
    return (
        test_dataset
        # Tokenize each samples using MobileBERT tokenizer
        #.map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=20, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly TEST_BATCH_SIZE
        # and make the shape **exactly** (TEST_BATCH_SIZE, SEQ_LENGTH)
        .batch(EXPERIMENT_CONFIG['TEST_BATCH_SIZE'])#, drop_remainder=True)
    )   

### Training set

In [18]:
# Since the dataset is pretty large, we randomly select TRAIN_NUM_CLIENT_LIMIT number of clients.
all_train_client_ids = train_client_data.client_ids

random.shuffle(all_train_client_ids)

if EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT'] > 0:
    selected_train_client_ids = all_train_client_ids[0:EXPERIMENT_CONFIG['TRAIN_NUM_CLIENT_LIMIT']]
else:
    selected_train_client_ids = all_train_client_ids

In [19]:
train_client_data = train_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [20]:
train_client_data = train_client_data.preprocess(preprocess_fn=preprocess_for_train)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [21]:
print(train_client_data.element_type_structure)

(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))


In [22]:
train_client_states = {}

# Initialize client states for all clients (selected for the entire simulation)
for i, client_id in enumerate(selected_train_client_ids):
    train_client_states[client_id] = fedavg_client.ClientState(client_serial=i, visit_count=0)

### Test set

In [23]:
test_client_data_all_merged = test_client_data.create_tf_dataset_for_client(
    test_client_data.client_ids[0]).filter(check_empty_snippet)

if len(test_client_data.client_ids) > 1:
    for i in range(1, len(test_client_data.client_ids)):
        test_client_data_all_merged = test_client_data_all_merged.concatenate(
            test_client_data.create_tf_dataset_for_client(test_client_data.client_ids[i]).filter(check_empty_snippet))

In [24]:
test_client_data_all_merged = preprocess_for_test(test_client_data_all_merged)

In [25]:
test_client_data_all_merged = test_client_data_all_merged.cache()

In [26]:
test_client_data_all_merged = test_client_data_all_merged.prefetch(10000)

In [27]:
print(test_client_data_all_merged.element_spec)

(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))


## Model

In [28]:
bert_model = transformers.TFAutoModelForPreTraining.from_pretrained(
    EXPERIMENT_CONFIG['HUGGINGFACE_MODEL_NAME'], cache_dir=EXPERIMENT_CONFIG['HUGGINGFACE_CACHE_DIR'])

All model checkpoint layers were used when initializing TFMobileBertForPreTraining.

All the layers of TFMobileBertForPreTraining were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertForPreTraining for predictions without further training.


In [29]:
print(bert_model.config)

MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "trigram_input": true,
  "true_hidden_size": 128,
  "type_vocab_size": 2,
  "use_bottleneck": true,
  "use_bottleneck_attention": false,
  "vocab_size": 30522
}



In [30]:
# Due to the limitations with Keras subclasses, we can only use the main layer part from pretrained models
# and add output heads by ourselves
bert_keras_converted = utils.convert_huggingface_mlm_to_keras(
    huggingface_model=bert_model,
    max_seq_length=EXPERIMENT_CONFIG['BERT_MAX_SEQ_LENGTH'],
)

In [31]:
# Use lists of NumPy arrays to backup pretained weights
bert_pretrained_trainable_weights = []
bert_pretrained_non_trainable_weights = []

for w in bert_keras_converted.trainable_weights:
    bert_pretrained_trainable_weights.append(w.numpy())

for w in bert_keras_converted.non_trainable_weights:
    bert_pretrained_non_trainable_weights.append(w.numpy())

In [32]:
def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""

    loss = utils.MaskedLMCrossEntropy()

    model_wrapped = utils.KerasModelWrapper(
        tf.keras.models.clone_model(bert_keras_converted),
        train_client_data.element_type_structure, loss)

    return model_wrapped

## Training

### Training setups

In [33]:
summary_writer = tf.summary.create_file_writer(EXPERIMENT_CONFIG['RESULTS_LOG'])

In [34]:
def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['SERVER_LEARNING_RATE'])

def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=EXPERIMENT_CONFIG['CLIENT_LEARNING_RATE'])

In [35]:
%%time

iterative_process = fedavg.build_federated_averaging_process(
    model_fn=tff_model_fn,
    model_input_spec=train_client_data.element_type_structure,
    initial_trainable_weights=bert_pretrained_trainable_weights,
    initial_non_trainable_weights=bert_pretrained_non_trainable_weights,
    server_optimizer_fn=server_optimizer_fn, 
    client_optimizer_fn=client_optimizer_fn)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


NameError: in user code:

    /root/BERTerated/fedavg_client.py:99 update_client  *
        outputs = model.forward_pass(batch)
    /root/BERTerated/utils.py:68 forward_pass  *
        if tf_device_identifier is not None:

    NameError: name 'tf_device_identifier' is not defined


In [37]:
%%time

server_state = iterative_process.initialize()

In [38]:
metric_eval = tfa.metrics.MeanMetricWrapper(fn=utils.calculate_masked_lm_cross_entropy, name='ce')

In [39]:
# The model for calculating validation loss only
# (This happens outside FedAvg)
model_final = utils.KerasModelWrapper(
    tf.keras.models.clone_model(bert_keras_converted),
    train_client_data.element_type_structure,
    utils.MaskedLMCrossEntropy(),
    tf_device_identifier="/GPU:0")

### Training loop

In [40]:
%%time

with summary_writer.as_default():
    for round_num in range(1, EXPERIMENT_CONFIG['TOTAL_ROUNDS'] + 1):        
        # FedAvg
        print(f'Round {round_num} start!')

        # Training client selection
        sampled_client_serials = np.random.choice(
            len(selected_train_client_ids),
            size=EXPERIMENT_CONFIG['TRAIN_CLIENTS_PER_ROUND'],
            replace=False)

        sampled_train_data = [
            train_client_data.create_tf_dataset_for_client(selected_train_client_ids[client_serial])
            for client_serial in sampled_client_serials
        ]
        
        sampled_client_states = [
            train_client_states[selected_train_client_ids[client_serial]]
            for client_serial in sampled_client_serials
        ]
        
        print("Selected client serials:", sampled_client_serials)

        current_round_start_time = time.time()
        
        server_state, new_client_states, train_loss = iterative_process.next(
            server_state, sampled_client_states, sampled_train_data)
        
        current_round_end_time = time.time()
        
        currnt_round_running_time = current_round_end_time - current_round_start_time

        print(f'Round {round_num} training loss: {train_loss}')
        print(f'Round {round_num} execution time: {currnt_round_running_time}')
        
        # Record the current round's training loss to the log
        tf.summary.scalar('train_loss', train_loss, step=round_num)
        tf.summary.scalar('train_running_time', currnt_round_running_time, step=round_num)
        
        print()
        
        # Update client states
        print("Updating client states.")

        for state in new_client_states:
            train_client_states[selected_train_client_ids[state.client_serial]] = state

        print()

        # Evaluation
        if round_num % EXPERIMENT_CONFIG['ROUNDS_PER_EVAL'] == 0:
            model_final.from_weights(server_state.model_weights)

            # Test dataset generation for this round
            print("Calculating validation metric:")

            current_round_validation_start_time = time.time()
            
            current_round_validation_metric = utils.keras_evaluate(
                model_final.keras_model, test_client_data_all_merged, metric_eval, "/GPU:0")
            
            current_round_validation_end_time = time.time()
            
            current_round_validation_runnning_time = current_round_validation_end_time - current_round_validation_start_time

            print(f'Round {round_num} validation metric: {current_round_validation_metric}')
            print(f'Round {round_num} validation time: {current_round_validation_runnning_time}')
            
            # Write down train_metrics to the log
            tf.summary.scalar('validation_metric', current_round_validation_metric, step=round_num)
            tf.summary.scalar('validation_running_time', current_round_validation_runnning_time, step=round_num)
            
        print()

Round 1 start!
Selected client serials: [267 191 488 146  82 205 384  38  86  78]
Anonymous client 78 : updated the model with server message.
Anonymous client 78 : training start.
Anonymous client 78 : batch 1 , 8 examples processed
Anonymous client 78 : batch 2 , 16 examples processed
Anonymous client 78 : batch 3 , 19 examples processed
Anonymous client 86 : updated the model with server message.
Anonymous client 86 : training start.
Anonymous client 86 : batch 1 , 8 examples processed
Anonymous client 78 : batch 4 , 27 examples processed
Anonymous client 86 : batch 2 , 16 examples processed
Anonymous client 82 : updated the model with server message.
Anonymous client 82 : training start.
Anonymous client 78 : batch 5 , 35 examples processed
Anonymous client 82 : batch 1 , 8 examples processed
Anonymous client 86 : batch 3 , 24 examples processed
Anonymous client 78 : batch 6 , 38 examples processed
Anonymous client 82 : batch 2 , 16 examples processed
Anonymous client 78 : batch 7 

Anonymous client 232 : batch 1 , 6 examples processed
Anonymous client 660 : batch 1 , 1 examples processed
Anonymous client 547 : batch 1 , 8 examples processed
Anonymous client 295 : batch 1 , 8 examples processed
Anonymous client 161 : batch 2 , 2 examples processed
Anonymous client 629 : batch 1 , 8 examples processed
Anonymous client 292 : batch 1 , 8 examples processed
Anonymous client 714 : batch 1 , 8 examples processed
Anonymous client 660 : batch 2 , 2 examples processed
Anonymous client 647 : batch 2 , 2 examples processed
Anonymous client 161 : batch 3 , 3 examples processed
Anonymous client 601 : batch 2 , 4 examples processed
Anonymous client 660 : batch 3 , 3 examples processed
Anonymous client 647 : batch 3 , 3 examples processed
Anonymous client 161 : training finished. 3  examples processed, loss: 10.3252401
Anonymous client 660 : training finished. 3  examples processed, loss: 10.3244925
Anonymous client 647 : training finished. 3  examples processed, loss: 10.319808

Selected client serials: [412 171 168 521 570 185 415 389 218 394]
Anonymous client 389 : updated the model with server message.
Anonymous client 389 : training start.
Anonymous client 171 : updated the model with server message.
Anonymous client 171 : training start.
Anonymous client 412 : updated the model with server message.
Anonymous client 412 : training start.
Anonymous client 570 : updated the model with server message.
Anonymous client 570 : training start.
Anonymous client 218 : updated the model with server message.
Anonymous client 218 : training start.
Anonymous client 185 : updated the model with server message.
Anonymous client 185 : training start.
Anonymous client 394 : updated the model with server message.
Anonymous client 394 : training start.
Anonymous client 521 : updated the model with server message.
Anonymous client 521 : training start.
Anonymous client 168 : updated the model with server message.
Anonymous client 389 : batch 1 , 8 examples processed
Anonymous

Anonymous client 171 : batch 26 , 198 examples processed
Anonymous client 412 : batch 27 , 215 examples processed
Anonymous client 389 : batch 26 , 204 examples processed
Anonymous client 171 : batch 27 , 206 examples processed
Anonymous client 412 : batch 28 , 223 examples processed
Anonymous client 389 : batch 27 , 212 examples processed
Anonymous client 171 : batch 28 , 214 examples processed
Anonymous client 412 : batch 29 , 231 examples processed
Anonymous client 389 : batch 28 , 220 examples processed
Anonymous client 171 : batch 29 , 222 examples processed
Anonymous client 412 : batch 30 , 239 examples processed
Anonymous client 389 : batch 29 , 228 examples processed
Anonymous client 171 : batch 30 , 230 examples processed
Anonymous client 412 : batch 31 , 247 examples processed
Anonymous client 389 : batch 30 , 236 examples processed
Anonymous client 171 : batch 31 , 238 examples processed
Anonymous client 412 : batch 32 , 255 examples processed
Anonymous client 389 : batch 31

Anonymous client 209 : batch 16 , 126 examples processed
Anonymous client 209 : batch 17 , 134 examples processed
Anonymous client 209 : batch 18 , 141 examples processed
Anonymous client 209 : training finished. 141  examples processed, loss: 10.3177071
Round 4 training loss: 10.317360877990723

Updating client states.

Calculating validation metric:
Round 4 validation metric: 10.316728591918945

Round 5 start!
Selected client serials: [601 562  23 148 221 670  40 202 566  11]
Anonymous client 601 : updated the model with server message.
Anonymous client 601 : training start.
Anonymous client 202 : updated the model with server message.
Anonymous client 202 : training start.
Anonymous client 11 : updated the model with server message.
Anonymous client 11 : training start.
Anonymous client 670 : updated the model with server message.
Anonymous client 670 : training start.
Anonymous client 221 : updated the model with server message.
Anonymous client 221 : training start.
Anonymous clie

...

Anonymous client 433 : batch 1 , 8 examples processed
Anonymous client 616 : batch 1 , 8 examples processed
Anonymous client 453 : batch 1 , 8 examples processed
Anonymous client 578 : batch 1 , 8 examples processed
Anonymous client 414 : batch 1 , 8 examples processed
Anonymous client 110 : batch 1 , 7 examples processed
Anonymous client 462 : batch 1 , 1 examples processed
Anonymous client 509 : batch 2 , 2 examples processed
Anonymous client 462 : batch 2 , 2 examples processed
Anonymous client 509 : batch 3 , 3 examples processed
Anonymous client 641 : batch 2 , 9 examples processed
Anonymous client 462 : batch 3 , 3 examples processed
Anonymous client 509 : training finished. 3  examples processed, loss: 10.320075
Anonymous client 353 : batch 2 , 16 examples processed
Anonymous client 616 : batch 2 , 16 examples processed
Anonymous client 462 : training finished. 3  examples processed, loss: 10.3039675
Anonymous client 433 : batch 2 , 16 examples processed
Anonymous client 453 : b

Anonymous client 660 : batch 1 , 1 examples processed
Anonymous client 539 : batch 1 , 8 examples processed
Anonymous client 638 : batch 1 , 4 examples processed
Anonymous client 596 : batch 1 , 7 examples processed
Anonymous client 714 : batch 1 , 8 examples processed
Anonymous client 567 : batch 1 , 5 examples processed
Anonymous client 315 : batch 1 , 8 examples processed
Anonymous client 660 : batch 2 , 2 examples processed
Anonymous client 176 : batch 2 , 6 examples processed
Anonymous client 394 : batch 2 , 10 examples processed
Anonymous client 660 : batch 3 , 3 examples processed
Anonymous client 169 : batch 2 , 16 examples processed
Anonymous client 660 : training finished. 3  examples processed, loss: 10.3108778
Anonymous client 638 : batch 2 , 8 examples processed
Anonymous client 567 : batch 2 , 10 examples processed
Anonymous client 539 : batch 2 , 16 examples processed
Anonymous client 176 : batch 3 , 9 examples processed
Anonymous client 596 : batch 2 , 14 examples proce

Anonymous client 659 : batch 8 , 56 examples processed
Anonymous client 368 : batch 8 , 64 examples processed
Anonymous client 430 : batch 9 , 72 examples processed
Anonymous client 385 : batch 8 , 64 examples processed
Anonymous client 430 : batch 10 , 80 examples processed
Anonymous client 659 : batch 9 , 60 examples processed
Anonymous client 368 : batch 9 , 72 examples processed
Anonymous client 385 : batch 9 , 72 examples processed
Anonymous client 659 : training finished. 60  examples processed, loss: 10.3095465
Anonymous client 385 : batch 10 , 80 examples processed
Anonymous client 368 : batch 10 , 80 examples processed
Anonymous client 430 : batch 11 , 88 examples processed
Anonymous client 385 : batch 11 , 88 examples processed
Anonymous client 368 : batch 11 , 88 examples processed
Anonymous client 430 : batch 12 , 96 examples processed
Anonymous client 385 : batch 12 , 96 examples processed
Anonymous client 368 : batch 12 , 96 examples processed
Anonymous client 430 : batch

Anonymous client 698 : batch 5 , 40 examples processed
Anonymous client 513 : batch 6 , 48 examples processed
Anonymous client 204 : batch 6 , 48 examples processed
Anonymous client 204 : training finished. 48  examples processed, loss: 10.3037682
Anonymous client 636 : batch 6 , 42 examples processed
Anonymous client 513 : batch 7 , 51 examples processed
Anonymous client 698 : batch 6 , 48 examples processed
Anonymous client 369 : batch 6 , 36 examples processed
Anonymous client 636 : batch 7 , 50 examples processed
Anonymous client 698 : training finished. 48  examples processed, loss: 10.3094893
Anonymous client 369 : batch 7 , 44 examples processed
Anonymous client 513 : batch 8 , 59 examples processed
Anonymous client 369 : batch 8 , 52 examples processed
Anonymous client 636 : batch 8 , 58 examples processed
Anonymous client 513 : batch 9 , 67 examples processed
Anonymous client 369 : batch 9 , 54 examples processed
Anonymous client 636 : batch 9 , 63 examples processed
Anonymous

ResourceExhaustedError: OOM when allocating tensor with shape[32,128,30522] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:AddV2]

### Save the trained model

In [ ]:
model_final.keras_model.save(EXPERIMENT_CONFIG['RESULTS_MODEL'])

## Evaluation

In [ ]:
test_metric = utils.keras_evaluate(
    model_final.keras_model, test_client_data_all_merged, metric_eval)

print(test_metric)

In [ ]:
# Debug: Did the client states get updated correctly?
print(train_client_states)